In [1]:
%cd ..

import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config

import smtplib
import base64
from email.mime.text import MIMEText
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.utils import COMMASPACE
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
import os
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl
import networkx as nx
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from IPython.display import display

/home/ec2-user/May


/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


('Reading config file from location: '
 '/home/ec2-user/May/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/May/followers',
                    'friends': '/home/ec2-user/May/friends'},
          'cwd': '/home/ec2-user/May/data/givenchy',
          'ml': '/home/ec2-user/May/data/givenchy/pickle',
          'networkx': {'all': '/home/ec2-user/May/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/May/data/givenchy/pickle/networkx_friends.dat',
                       'potential': '/home/ec2-user/May/data/givenchy/pickle/networkx_potential.dat'},
         

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

path = "/home/ec2-user/May/data/givenchy/pickle"  

def write_to_pickle(file_path, data):

    bytes_out = pickle.dumps(data)
    with open(file_path, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    return pickle.loads(bytes_in)

def loadModel(self):
    read_file = "data/" + self.model_name + ".dat"
    self.model = joblib.load(read_file)
    return self.model


# Load data

In [3]:
model = joblib.load(path+"/xgb_model_GIVENCHY (2).dat")
unique_users = load_pickle(path+"/80000users_3.pkl")
network = load_pickle(path+"/80000users_3.pkl")
users=load_pickle(path+'/Stanford_features.pkl')
betweenness =pd.read_csv('/home/ec2-user/May/data/centrality.csv')

In [4]:
unique_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70013 entries, 0 to 70012
Data columns (total 14 columns):
user_favourites_count               70013 non-null int64
followers_count                     70013 non-null int64
friends_count                       70013 non-null int64
user_id                             70013 non-null int64
user_listed_count                   70013 non-null int64
user_statuses_count                 70013 non-null int64
source_candidates                   70013 non-null object
infec_candidates                    70013 non-null object
user_created_days                   70013 non-null int64
normalized_user_statuses_count      70013 non-null float64
normalized_user_followers_count     70013 non-null float64
normalized_user_favourites_count    70013 non-null float64
normalized_user_listed_count        70013 non-null float64
normalized_user_friends_count       70013 non-null float64
dtypes: float64(5), int64(7), object(2)
memory usage: 7.5+ MB


In [5]:
# ######## infec_candidates ########
# temp=[]
# infec = [temp for i in range(len(unique_users))]
# unique_users["infec_candidates"]= None
# record_num = int(network.describe().iloc[0,0])

# for i in range(record_num):
#     source_candidates = unique_users.loc[i,'source_candidates']
#     #users.loc[i,"infec_candidates"] = list()
#     for j in range(len(source_candidates)):
#         candidates = source_candidates[j]
#         if infec[candidates] == []:
#             infec[candidates]=[i]
#         else :
#             infec[candidates]+=[i]

# unique_users.loc[:,"infec_candidates"]=infec

# unique_users.head()

# reprocess_data

In [6]:
def reprocess_data(time,origins,infectors,users,unique_users):
        i= int(users[users.user_id.isin([origins])].index.values)
        j= int(users[users.user_id.isin([infectors])].index.values)
        source_candidates = unique_users.loc[j,'source_candidates']
        sources = [x for x in source_candidates if users.loc[x,'TwM_time']<=time]
        sources_time=users[users.index.isin(sources)].sort_values(by=['TwM_time'])
        sources_dataframe = unique_users.loc[sources]
        timeList =  [users.loc[x].TwM_tCurrent - users.loc[x].TwM_time for x in sources]
        degreeList = list(unique_users.loc[k, 'followers_count'] + unique_users.loc[k, 'friends_count']  for k in sources)
    
            
        m=sources_time.index.values[0]
        n=sources_time.index.values[-1]
        
        users.loc[j,'UsM_deltaDays0'] = users.loc[m,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[m,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0'] = users.loc[m,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[m,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[m,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[m,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[m,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[m,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[m,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[m,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[n,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[n,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[n,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[n,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[n,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[n,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[n,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[n,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[n,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[n,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[n,'UsM_normalizedUserFriendsCount']
        
       
        users.loc[j,'TwM_t0'] = users.loc[m].TwM_tCurrent - users.loc[m,'TwM_time']
        users.loc[j,'TwM_tSeed0'] = users.loc[m,'TwM_tSeed0']
        users.loc[j,'TwM_t-1'] = users.loc[m].TwM_tCurrent - users.loc[n,'TwM_time']
        users.loc[j,'TwM_tSeed-1'] = users.loc[n,'TwM_tSeed-1']
        users.loc[j,'TwM_tCurrent'] =  np.ceil(time/30)*30
        
        
        users.loc[j,'Nw_degree0'] = users.loc[m,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[n,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[m,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[n,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[n,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[m,'Nw_degreeSeed0']
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[m,'Nw_inDegreeSeed0']
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[m,'Nw_outDegreeSeed0']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[n,'Nw_degreeSeed-1']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[n,'Nw_inDegreeSeed-1']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[n,'Nw_outDegreeSeed-1']
        
        
        
        users.loc[j,'SNw_generation0'] = users.loc[m,'SNw_generation']
        users.loc[j,'SNw_generation-1'] = users.loc[n,'SNw_generation']
        users.loc[j,'SNw_timeSinceSeed0'] = users.loc[m,'SNw_timeSinceSeed0']
        users.loc[j,'SNw_timeSinceSeed-1'] = users.loc[n,'SNw_timeSinceSeed-1']
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
        users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = safe_division(in_degree[j], users.loc[j,'SNw_totalInDegree'])
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[m,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[n,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = safe_division(out_degree[j], users.loc[j,'SNw_totalOutDegree'])
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[m,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[n,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[m,'SNw_inDegreeCentralitySeed0']
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[m,'SNw_outDegreeCentralitySeed0']
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[n,'SNw_inDegreeCentralitySeed-1']
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[n,'SNw_outDegreeCentralitySeed-1']


        users.loc[j,'SNw_nFriendsInfected']=len(sources)
        users.loc[j,'SNw_friendsInfectedRatio']=safe_division(len(sources), unique_users.loc[j,'friends_count'])
            
        users.loc[j,'Stat_average_kOut']=round(mean(degreeList), 1)
        users.loc[j,'Stat_average_t']=round(mean(timeList), 1)
        users.loc[j,'Stat_average_deltaDays']=sources_dataframe.user_created_days.mean()
        users.loc[j,'Stat_average_statusesCount']=sources_dataframe.user_statuses_count.mean()
        users.loc[j,'Stat_average_followersCount']=sources_dataframe.followers_count.mean()
        users.loc[j,'Stat_average_favouritesCount']=sources_dataframe.user_favourites_count.mean()
        users.loc[j,'Stat_average_friendsCount']=sources_dataframe.friends_count.mean()
        users.loc[j,'Stat_average_listedCount']=sources_dataframe.user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserStatusesCount']=sources_dataframe.normalized_user_statuses_count.mean()
        users.loc[j,'Stat_average_normalizedUserFollowersCount']=sources_dataframe.normalized_user_followers_count.mean()
        users.loc[j,'Stat_average_normalizedUserFavouritesCount']=sources_dataframe.normalized_user_favourites_count.mean()
        users.loc[j,'Stat_average_normalizedUserListedCount']=sources_dataframe.normalized_user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserFriendsCount']=sources_dataframe.normalized_user_friends_count.mean()
        if len(degreeList)!=0 :
            users.loc[j,'Stat_max_kOut']=max(degreeList) 
            users.loc[j,'Stat_min_kOut']=min(degreeList) 
        else:
            users.loc[j,'Stat_max_kOut']=''
            users.loc[j,'Stat_min_kOut']=''
#         temp = users[users.index.isin(['j'])]
        return users

In [7]:
# intervals = [60, 30, 15, 7]
intervals = [30]

# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y
    
# infectedNetworkStats = {}
# for interval in intervals:
#     statsDictionary = {}
#     for timestamp in range(0, 24*60+1, interval):
#         infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
#         statsDictionary[timestamp] = {}
#         statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
#         statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
#         statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
#     infectedNetworkStats[interval] = statsDictionary

def infect_totalNodeInfected(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = infected_dataframe.shape[0]
    return states

def infect_totalFollowers(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_followersCount)
    return states

def infect_totalFriends(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_friendsCount)
    return states

in_degree = list(unique_users.friends_count)
out_degree = list(unique_users.followers_count)
degree = [sum(x) for x in zip(in_degree, out_degree)]

In [8]:
network_simulation = pd.DataFrame(columns=["t_current","user_id","source_candidates","infec_candidates",
                                           "followers_count","friends_count","source_id","seed_user_id",
                                           "SIR","generations","time_lapsed","time_since_seed"])
# network_simulation["user"] =unique_users["user"]
network_simulation["user_id"] = unique_users["user_id"]
network_simulation["source_candidates"] = unique_users["source_candidates"]
network_simulation["infec_candidates"] = unique_users["infec_candidates"]
network_simulation["followers_count"] = unique_users["followers_count"]
network_simulation["friends_count"] = unique_users["friends_count"]
#network_simulation.head()
# network_simualtion = load_pickle(path+'/Stanford_network_simulation_0.6.pkl')

In [9]:
##situation choose:
##a.全部为0   b.从0开始由小到大    c.从0开始由大到小

betweenness=betweenness.sort_values(by='betweenesscentrality')
betweenness=betweenness.iloc[len(betweenness)-5:len(betweenness)]
# betweenness
between_id =[x for x in betweenness['Id']]

seeds = network_simulation[network_simulation.user_id.isin(between_id)]
seeds['time_lapsed']=0
seeds

/home/ec2-user/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,t_current,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed
451,NaN,15439395,"[479, 510, 482, 541, 493, 495, 507, 580, 536, ...","[452, 528, 490, 483, 517, 1198, 1156, 1312, 36...",12754295,50151,NaN,NaN,NaN,NaN,0,NaN
806,NaN,3359851,"[797, 834, 800, 819, 848, 822, 793, 829, 820, ...","[789, 828, 790, 799, 835, 791, 830, 796, 824, ...",503906,27988,NaN,NaN,NaN,NaN,0,NaN
1441,NaN,813286,"[1415, 1485, 1434, 1426, 1535, 1460, 1459, 153...","[1490, 1504, 1489, 758, 771, 1495, 4491, 4509,...",101454968,620510,NaN,NaN,NaN,NaN,0,NaN
2242,NaN,17093617,"[2241, 2261, 2220, 2277, 2270, 2297, 2319, 225...","[2212, 2229, 2243, 2285, 2218, 2362, 2213, 226...",7966561,1517036,NaN,NaN,NaN,NaN,0,NaN
10261,NaN,15846407,"[10143, 10123, 10128, 10199, 10233, 10192, 102...","[1441, 10238, 10169, 10245, 10279, 10306, 1032...",75965729,35685,NaN,NaN,NaN,NaN,0,NaN


In [10]:
### seeds ###
def seeds_setting_simulation(seeds,network_simulation):
    for i in range(len(seeds)):
        index = seeds.index.values
        network_simulation.loc[index,'SIR'] = 1
        network_simulation.loc[index,'generations'] = 0
        network_simulation.loc[index,'source_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_lapsed'] = seeds.loc[index,'time_lapsed']
        network_simulation.loc[index,'seed_user_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_since_seed'] = 0
    return network_simulation

def seeds_setting_features(seeds,users):
    for i in range(len(seeds)):

   
        j = seeds.index.values[i]
#         sources = [x for x in seeds.loc[j,'source_candidates']]
        users.loc[j,'UsM_deltaDays0'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'TwM_t0'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_t-1'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_tCurrent'] = np.ceil(seeds.loc[j,'time_lapsed']+0.001/30)*30   
        users.loc[j,'TwM_tSeed0'] = seeds.loc[j,'time_lapsed']                         
        users.loc[j,'TwM_tSeed-1'] = seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_time'] = seeds.loc[j,'time_lapsed']

        users.loc[j,'Nw_degree0'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[j,'Nw_degree']    
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[j,'UsM_friendsCount'] 
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[j,'UsM_followersCount']
        
        users.loc[j,'SNw_generation0'] = 0
        users.loc[j,'SNw_generation-1'] = 0
        users.loc[j,'SNw_generation'] = 0
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
#         users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = safe_division(in_degree[j], users.loc[j,'SNw_totalInDegree'])
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = safe_division(out_degree[j], users.loc[j,'SNw_totalOutDegree'])
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[j,'SNw_inDegreeCentrality']               
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[j,'SNw_outDegreeCentrality']                
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[j,'SNw_inDegreeCentrality']                  
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[j,'SNw_outDegreeCentrality']
    return users

In [11]:
seeds_setting_simulation(seeds,network_simulation)
seeds_setting_features(seeds,users)

,label,UsM_deltaDays,UsM_statusesCount,UsM_followersCount,UsM_favouritesCount,UsM_friendsCount,UsM_listedCount,UsM_normalizedUserStatusesCount,UsM_normalizedUserFollowersCount,UsM_normalizedUserFavouritesCount,...,Stat_average_friendsCount,Stat_average_listedCount,Stat_average_normalizedUserStatusesCount,Stat_average_normalizedUserFollowersCount,Stat_average_normalizedUserFavouritesCount,Stat_average_normalizedUserListedCount,Stat_average_normalizedUserFriendsCount,Stat_max_kOut,Stat_min_kOut,user_id
0,NaN,2679,201,2090,548,46,65,0.075028,0.780142,0.204554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214328887
1,NaN,3246,5695,305257,38,181,4929,1.75447,94.041,0.0117067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34428380
2,NaN,3417,5124,11756,31,498,93,1.49956,3.44044,0.00907229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17116707
3,NaN,3266,35499,57992,4449,1482,894,10.8693,17.7563,1.36222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28465635
4,NaN,2359,20720,15033,1968,219,45,8.78338,6.37262,0.834252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380580781
5,NaN,3344,14602,284075,2158,68,1934,4.36663,84.9507,0.645335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18996905
6,NaN,2660,6933,2121,309,206,12,2.60639,0.797368,0.116165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221036078
7,NaN,2834,807,4710,74,35,49,0.284757,1.66196,0.0261115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153460275
8,NaN,2970,3021,11627,350,934,63,1.01717,3.91481,0.117845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107830991
9,NaN,3385,4153,16996,9,552,238,1.22688,5.02097,0.00265879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17868918


In [12]:
network_simulation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70013 entries, 0 to 70012
Data columns (total 12 columns):
t_current            0 non-null object
user_id              70013 non-null int64
source_candidates    70013 non-null object
infec_candidates     70013 non-null object
followers_count      70013 non-null int64
friends_count        70013 non-null int64
source_id            5 non-null object
seed_user_id         5 non-null object
SIR                  5 non-null object
generations          5 non-null object
time_lapsed          5 non-null object
time_since_seed      5 non-null object
dtypes: int64(3), object(9)
memory usage: 6.4+ MB


In [15]:

def simulation(t,unique_users,network_simulation):
    for i in range(len(unique_users)):
        if np.isnan(network_simulation.loc[i,"time_lapsed"]) == False :
            if round(network_simulation.loc[i,"time_lapsed"]) <= t:
                infec_candidates=network_simulation.loc[i,"infec_candidates"]

                for j in range(len(infec_candidates)):
                    infector = network_simulation.loc[i,"infec_candidates"][j]

                    if network_simulation.loc[infector,'SIR']!=1 :
                        user_id1 = network_simulation.loc[i,'user_id']
                        user_id2 = network_simulation.loc[infector,'user_id']
                        processed_user=reprocess_data(t+1,user_id1,user_id2,users,unique_users)

                        valid = processed_user[processed_user.index.isin([infector])]
#                             print(valid['Stat_average_t'])
                        valid = valid.drop(['label','user_id','SNw_generation','TwM_time'],axis=1)

                        valid = valid.astype('float64')

                        columns = list(valid.columns)
                        X = valid[columns]
                        pre_data = xgb.DMatrix(X)
                        infec = model.predict(pre_data)
#                         print(infec)
                        if infec>0.8 :
                            users.loc[infector,'TwM_time'] = t
                            network_simulation.loc[infector,"SIR"] = 1
                            network_simulation.loc[infector,"generations"] = network_simulation.loc[i,"generations"]+1
                            network_simulation.loc[infector,"source_id"] = network_simulation.loc[i,"user_id"] 
                            network_simulation.loc[infector,"seed_user_id"] = network_simulation.loc[i,"seed_user_id"] 
                            network_simulation.loc[infector,"time_lapsed"] = t+1
            #             network_simulation.loc[i,"SIR"]=2 
    return unique_users,network_simulation



In [ ]:
##time = 5*60 minutes with resuming, due to releasing memory
for t in range(0,6):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Stanford_network_simulation_betweenness_5.pkl', network_simulation)

# for t in range(10,20):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(20,30):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(30,40):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(40,50):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(50,60):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(70,80):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(90,100):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(110,120):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(120,130):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(130,140):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(140,150):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(150,160):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(160,170):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(170,180):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(180,190):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(190,200):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(210,220):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(220,230):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(230,240):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(240,250):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(250,260):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(260,270):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(270,280):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(280,290):
#     temp_data=simulation(t,unique_users,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

# for t in range(290,300):
#     temp_data=simulation(t,unique_us01ers,network_simulation)
#     unique_users = temp_data[0]
#     network_simulation = temp_data[1]
#     print(t)
#     write_to_pickle(path+'/Stanford_network_simulation_0.6.pkl', network_simulation)

0
1
2


In [ ]:
network_simulation["t_current"]=users["TwM_tCurrent"]
# network_simulation

In [19]:
# network_simulation[network_simulation.generations.isin([0])]
x=[0,1,2,3,4,5]
network_simulation[network_simulation.generations.isin([5])]

# network_simulation.info()

,t_current,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed
8,NaN,107830991,"[10, 61, 35, 28, 41, 56, 52, 3, 49, 113, 69, 9...","[9, 58, 20, 77, 54, 6, 31, 43, 165, 21, 153, 3...",11627,934,151338729,3359851,1,5,5,NaN
25,NaN,280935165,"[20, 96, 18, 58, 111, 156, 69, 90, 59, 38, 76,...","[26, 42, 44, 122, 165, 72, 32, 70, 94, 22, 13,...",32706,124,314316607,3359851,1,5,5,NaN
35,NaN,67864340,"[10, 117, 135, 82, 15, 18, 97, 44, 37, 84, 62,...","[36, 39, 79, 41, 4, 8, 56, 13, 81, 2, 93, 38, ...",6794,211,151338729,3359851,1,5,5,NaN
37,NaN,270449528,"[47, 113, 14, 97, 42, 103, 43, 87, 6, 112, 22,...","[38, 32, 36, 9, 39, 96, 127, 93, 16, 44, 94, 6...",30427,972,151338729,3359851,1,5,5,NaN
46,NaN,206923844,"[52, 77, 105, 156, 60, 113, 138, 96, 62, 18, 3...","[47, 51, 149, 55, 145, 6, 137, 192, 59, 70, 93...",19913,89,151338729,3359851,1,5,5,NaN
148,NaN,265077741,"[113, 61, 62, 10, 88, 173, 2, 83, 70, 90, 111,...","[94, 6, 72, 33, 58, 3, 34, 0, 32, 122, 135, 27...",637,715,151338729,3359851,1,5,5,NaN
165,NaN,204140367,"[25, 21, 8, 83, 173, 111, 140, 103, 47, 163, 9...","[86, 42, 41, 74, 34, 127, 75, 28, 27, 26, 69, ...",13881,935,151338729,3359851,1,5,5,NaN
192,NaN,273149543,"[46, 97, 76, 137, 86, 18, 54, 105, 156, 111, 1...",[],1267,0,151338729,3359851,1,5,5,NaN
195,NaN,72680406,"[222, 200, 214, 204, 231, 202, 229, 230]","[196, 224, 209, 254, 240, 255, 203, 249]",12676,462,83876527,15439395,1,5,5,NaN
205,NaN,22689667,"[204, 244, 202, 268, 235, 196, 230, 255, 238, ...","[264, 220, 240, 201, 203, 225, 211, 224, 54986...",3489,429,83876527,15439395,1,5,5,NaN


In [ ]:
write_to_pickle(path+'/Stanford_network_simulation_betweenness_5.pkl', network_simulation)

In [ ]:
# network_simulation